# NumPy vs TensorFlow: Direct Comparison

## 🎯 Learning Goals
- Compare your existing NumPy neural network with TensorFlow equivalent
- Understand when to use each approach
- See how TensorFlow simplifies complex operations
- Learn the trade-offs between manual and automatic implementations

## 📚 Prerequisites
- Completed your NumPy neural network notebooks
- Basic understanding of TensorFlow from previous notebook
- Familiarity with forward pass and backpropagation concepts

In [ ]:
# Import all necessary libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"NumPy version: {np.__version__}")
print(f"TensorFlow version: {tf.__version__}")

## 🔧 Your NumPy Implementation (Baseline)

Let's start with your existing NumPy neural network implementation:

In [ ]:
# Your NumPy Neural Network (from your existing implementation)
class NumPyNeuralNetwork:
    def __init__(self):
        """Initialize with the same structure as your existing network: 2→2→1"""
        # Initialize weights randomly (like your trainable version)
        self.weights_input_to_hidden = np.random.uniform(-1, 1, (2, 2))
        self.weights_hidden_to_output = np.random.uniform(-1, 1, (2, 1))
        self.bias_hidden = np.zeros((1, 2))
        self.bias_output = np.zeros((1, 1))
        
    def sigmoid(self, x):
        """Your familiar sigmoid activation function"""
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))  # Clipping for numerical stability
    
    def sigmoid_derivative(self, x):
        """Derivative of sigmoid for backpropagation"""
        s = self.sigmoid(x)
        return s * (1 - s)
    
    def forward_pass(self, inputs):
        """Forward pass through the network"""
        if inputs.ndim == 1:
            inputs = inputs.reshape(1, -1)
        
        # Input to Hidden Layer
        self.hidden_input = np.dot(inputs, self.weights_input_to_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)
        
        # Hidden to Output Layer
        self.output_input = np.dot(self.hidden_output, self.weights_hidden_to_output) + self.bias_output
        self.final_output = self.sigmoid(self.output_input)
        
        return self.final_output
    
    def backpropagation(self, inputs, target, learning_rate=0.1):
        """Your manual backpropagation implementation"""
        if inputs.ndim == 1:
            inputs = inputs.reshape(1, -1)
        if target.ndim == 1:
            target = target.reshape(1, -1)
        
        # Forward pass
        output = self.forward_pass(inputs)
        
        # Calculate output error
        output_error = output - target
        
        # Calculate gradients
        output_delta = output_error * self.sigmoid_derivative(self.output_input)
        hidden_error = np.dot(output_delta, self.weights_hidden_to_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_input)
        
        # Update weights
        self.weights_hidden_to_output -= learning_rate * np.dot(self.hidden_output.T, output_delta)
        self.weights_input_to_hidden -= learning_rate * np.dot(inputs.T, hidden_delta)
        
        # Update biases
        self.bias_output -= learning_rate * np.mean(output_delta, axis=0, keepdims=True)
        self.bias_hidden -= learning_rate * np.mean(hidden_delta, axis=0, keepdims=True)
        
        return np.mean(output_error ** 2)

# Create NumPy network
numpy_network = NumPyNeuralNetwork()
print("✅ NumPy Neural Network created")
print(f"Input→Hidden weights shape: {numpy_network.weights_input_to_hidden.shape}")
print(f"Hidden→Output weights shape: {numpy_network.weights_hidden_to_output.shape}")

## 🚀 TensorFlow Implementation (Modern Approach)

Now let's create the same network using TensorFlow's low-level API (similar to your NumPy approach):

In [ ]:
# TensorFlow Neural Network (Low-level, similar to your NumPy approach)
class TensorFlowNeuralNetwork:
    def __init__(self):
        """Initialize with same structure: 2→2→1"""
        # Initialize weights as Variables (trainable parameters)
        self.weights_input_to_hidden = tf.Variable(
            tf.random.uniform((2, 2), -1, 1), name='w1'
        )
        self.weights_hidden_to_output = tf.Variable(
            tf.random.uniform((2, 1), -1, 1), name='w2'
        )
        self.bias_hidden = tf.Variable(tf.zeros((1, 2)), name='b1')
        self.bias_output = tf.Variable(tf.zeros((1, 1)), name='b2')
    
    def forward_pass(self, inputs):
        """Forward pass using TensorFlow operations"""
        if len(inputs.shape) == 1:
            inputs = tf.expand_dims(inputs, 0)
        
        # Input to Hidden Layer
        hidden_input = tf.matmul(inputs, self.weights_input_to_hidden) + self.bias_hidden
        hidden_output = tf.nn.sigmoid(hidden_input)
        
        # Hidden to Output Layer
        output_input = tf.matmul(hidden_output, self.weights_hidden_to_output) + self.bias_output
        final_output = tf.nn.sigmoid(output_input)
        
        return final_output
    
    def train_step(self, inputs, target, learning_rate=0.1):
        """Training step with automatic differentiation"""
        if len(inputs.shape) == 1:
            inputs = tf.expand_dims(inputs, 0)
        if len(target.shape) == 1:
            target = tf.expand_dims(target, 0)
        
        with tf.GradientTape() as tape:
            # Forward pass
            predictions = self.forward_pass(inputs)
            # Calculate loss
            loss = tf.reduce_mean(tf.square(predictions - target))
        
        # Automatic gradient calculation!
        gradients = tape.gradient(loss, [
            self.weights_input_to_hidden,
            self.weights_hidden_to_output,
            self.bias_hidden,
            self.bias_output
        ])
        
        # Manual weight updates (like your NumPy version)
        self.weights_input_to_hidden.assign_sub(learning_rate * gradients[0])
        self.weights_hidden_to_output.assign_sub(learning_rate * gradients[1])
        self.bias_hidden.assign_sub(learning_rate * gradients[2])
        self.bias_output.assign_sub(learning_rate * gradients[3])
        
        return loss.numpy()

# Create TensorFlow network
tf_network = TensorFlowNeuralNetwork()
print("✅ TensorFlow Neural Network created")
print(f"Input→Hidden weights shape: {tf_network.weights_input_to_hidden.shape}")
print(f"Hidden→Output weights shape: {tf_network.weights_hidden_to_output.shape}")

## 🎯 Keras Implementation (High-Level Approach)

Now let's see how simple this becomes with Keras (TensorFlow's high-level API):

In [ ]:
# Keras Neural Network (High-level, super simple!)
def create_keras_network():
    """Create the same 2→2→1 network with Keras"""
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2, activation='sigmoid', input_shape=(2,)),  # Hidden layer
        tf.keras.layers.Dense(1, activation='sigmoid')                     # Output layer
    ])
    
    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
        loss='mse',  # Mean Squared Error
        metrics=['mse']
    )
    
    return model

# Create Keras network
keras_network = create_keras_network()
print("✅ Keras Neural Network created")
print("\nModel Summary:")
keras_network.summary()

print("\n💡 Notice how much simpler the Keras version is!")
print("   - No manual forward pass implementation")
print("   - No manual backpropagation")
print("   - No manual weight updates")
print("   - Everything is handled automatically!")

## 📊 Training Data Setup

Let's create some training data to compare all three approaches:

In [ ]:
# Create training data (same as your existing examples)
training_inputs = np.array([
    [1.0, 1.0],
    [0.0, 0.0],
    [1.0, 0.0],
    [0.0, 1.0]
])

training_outputs = np.array([
    [1.0],
    [0.0],
    [0.5],
    [0.5]
])

print("Training Data:")
for i in range(len(training_inputs)):
    print(f"Input: {training_inputs[i]} → Target: {training_outputs[i][0]}")

# Convert to TensorFlow tensors for TF networks
training_inputs_tf = tf.constant(training_inputs, dtype=tf.float32)
training_outputs_tf = tf.constant(training_outputs, dtype=tf.float32)

print(f"\nDataset size: {len(training_inputs)} samples")
print(f"Input shape: {training_inputs.shape}")
print(f"Output shape: {training_outputs.shape}")

## 🏃‍♂️ Training Comparison

Now let's train all three networks and compare their performance:

In [ ]:
# Training parameters
epochs = 1000
learning_rate = 0.1

# Storage for loss tracking
numpy_losses = []
tf_losses = []
keras_losses = []

print("🚀 Starting training comparison...")
print(f"Training for {epochs} epochs with learning rate {learning_rate}")

# Time the training processes
start_time = time.time()

# Train NumPy network
print("\n📊 Training NumPy Network...")
numpy_start = time.time()
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(training_inputs)):
        loss = numpy_network.backpropagation(
            training_inputs[i], 
            training_outputs[i], 
            learning_rate
        )
        total_loss += loss
    
    avg_loss = total_loss / len(training_inputs)
    numpy_losses.append(avg_loss)
    
    if epoch % 200 == 0:
        print(f"  Epoch {epoch}: Loss = {avg_loss:.6f}")

numpy_time = time.time() - numpy_start
print(f"NumPy training completed in {numpy_time:.3f} seconds")

# Train TensorFlow network
print("\n🔥 Training TensorFlow Network...")
tf_start = time.time()
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(training_inputs)):
        loss = tf_network.train_step(
            training_inputs_tf[i], 
            training_outputs_tf[i], 
            learning_rate
        )
        total_loss += loss
    
    avg_loss = total_loss / len(training_inputs)
    tf_losses.append(avg_loss)
    
    if epoch % 200 == 0:
        print(f"  Epoch {epoch}: Loss = {avg_loss:.6f}")

tf_time = time.time() - tf_start
print(f"TensorFlow training completed in {tf_time:.3f} seconds")

# Train Keras network
print("\n⚡ Training Keras Network...")
keras_start = time.time()
history = keras_network.fit(
    training_inputs_tf, 
    training_outputs_tf,
    epochs=epochs,
    verbose=0  # Silent training
)
keras_losses = history.history['loss']
keras_time = time.time() - keras_start
print(f"Keras training completed in {keras_time:.3f} seconds")

total_time = time.time() - start_time
print(f"\n⏱️ Total comparison time: {total_time:.3f} seconds")

## 📈 Results Visualization

Let's visualize the training progress and compare the results:

In [ ]:
# Plot training losses
plt.figure(figsize=(15, 5))

# Loss comparison
plt.subplot(1, 3, 1)
plt.plot(numpy_losses, 'b-', label='NumPy', linewidth=2)
plt.plot(tf_losses, 'r--', label='TensorFlow', linewidth=2)
plt.plot(keras_losses, 'g:', label='Keras', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training Loss Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')  # Log scale to see differences better

# Training time comparison
plt.subplot(1, 3, 2)
methods = ['NumPy', 'TensorFlow', 'Keras']
times = [numpy_time, tf_time, keras_time]
colors = ['blue', 'red', 'green']
bars = plt.bar(methods, times, color=colors, alpha=0.7)
plt.ylabel('Training Time (seconds)')
plt.title('Training Speed Comparison')
plt.grid(True, alpha=0.3)

# Add time labels on bars
for bar, time_val in zip(bars, times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{time_val:.3f}s', ha='center', va='bottom')

# Final loss comparison
plt.subplot(1, 3, 3)
final_losses = [numpy_losses[-1], tf_losses[-1], keras_losses[-1]]
bars = plt.bar(methods, final_losses, color=colors, alpha=0.7)
plt.ylabel('Final Loss (MSE)')
plt.title('Final Training Loss')
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Add loss labels on bars
for bar, loss_val in zip(bars, final_losses):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 1.1,
             f'{loss_val:.6f}', ha='center', va='bottom', rotation=45)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n📊 TRAINING SUMMARY")
print("=" * 50)
print(f"{'Method':<12} {'Time (s)':<10} {'Final Loss':<12} {'Speedup':<10}")
print("-" * 50)
print(f"{'NumPy':<12} {numpy_time:<10.3f} {numpy_losses[-1]:<12.6f} {'1.0x':<10}")
print(f"{'TensorFlow':<12} {tf_time:<10.3f} {tf_losses[-1]:<12.6f} {numpy_time/tf_time:<10.1f}x")
print(f"{'Keras':<12} {keras_time:<10.3f} {keras_losses[-1]:<12.6f} {numpy_time/keras_time:<10.1f}x")

## 🧪 Testing the Trained Networks

Let's test all three networks on the same inputs to see how they perform:

In [ ]:
# Test all networks on the training data
print("🧪 TESTING TRAINED NETWORKS")
print("=" * 60)
print(f"{'Input':<12} {'Target':<8} {'NumPy':<10} {'TensorFlow':<12} {'Keras':<10}")
print("-" * 60)

for i in range(len(training_inputs)):
    # Get predictions from all networks
    numpy_pred = numpy_network.forward_pass(training_inputs[i])[0][0]
    tf_pred = tf_network.forward_pass(training_inputs_tf[i]).numpy()[0][0]
    keras_pred = keras_network.predict(training_inputs[i:i+1], verbose=0)[0][0]
    
    target = training_outputs[i][0]
    
    print(f"{str(training_inputs[i]):<12} {target:<8.1f} {numpy_pred:<10.4f} {tf_pred:<12.4f} {keras_pred:<10.4f}")

# Test on some new data
print("\n🔍 TESTING ON NEW DATA")
print("=" * 60)
test_inputs = np.array([
    [0.5, 0.5],
    [0.8, 0.2],
    [0.3, 0.7]
])

print(f"{'Input':<12} {'NumPy':<10} {'TensorFlow':<12} {'Keras':<10}")
print("-" * 50)

for i in range(len(test_inputs)):
    numpy_pred = numpy_network.forward_pass(test_inputs[i])[0][0]
    tf_pred = tf_network.forward_pass(tf.constant(test_inputs[i])).numpy()[0][0]
    keras_pred = keras_network.predict(test_inputs[i:i+1], verbose=0)[0][0]
    
    print(f"{str(test_inputs[i]):<12} {numpy_pred:<10.4f} {tf_pred:<12.4f} {keras_pred:<10.4f}")

## 🔍 Code Complexity Comparison

Let's analyze the complexity of each approach:

In [ ]:
# Code complexity analysis
print("📝 CODE COMPLEXITY ANALYSIS")
print("=" * 50)

# Count lines of code for each implementation
numpy_lines = 65  # Approximate lines in NumPy class
tf_lines = 45     # Approximate lines in TensorFlow class  
keras_lines = 12  # Lines for Keras implementation

print(f"NumPy Implementation:    ~{numpy_lines} lines of code")
print(f"TensorFlow Implementation: ~{tf_lines} lines of code")
print(f"Keras Implementation:    ~{keras_lines} lines of code")

print("\n🧠 WHAT YOU NEED TO UNDERSTAND:")
print("-" * 40)
print("NumPy Approach:")
print("  ✅ Forward propagation math")
print("  ✅ Backpropagation algorithm")
print("  ✅ Gradient calculations")
print("  ✅ Weight update rules")
print("  ✅ Matrix operations")
print("  ✅ Activation functions")

print("\nTensorFlow Low-Level:")
print("  ✅ Forward propagation")
print("  ⚡ Automatic gradients")
print("  ✅ TensorFlow operations")
print("  ✅ Variable management")

print("\nKeras High-Level:")
print("  ⚡ Model architecture")
print("  ⚡ Compilation settings")
print("  ⚡ Training process")

print("\n💡 KEY INSIGHTS:")
print("-" * 40)
print(f"• NumPy: {numpy_lines/keras_lines:.1f}x more code, but full understanding")
print(f"• TensorFlow: {tf_lines/keras_lines:.1f}x more code, automatic gradients")
print(f"• Keras: Simplest, but abstracts away details")
print("• NumPy is best for learning fundamentals")
print("• Keras is best for rapid prototyping")
print("• TensorFlow low-level gives you control + automation")

## 🎯 When to Use Each Approach

Let's create a decision guide:

In [ ]:
# Create a visual decision guide
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Learning curve comparison
learning_stages = ['Beginner', 'Intermediate', 'Advanced', 'Expert']
numpy_difficulty = [3, 2, 1, 1]  # Gets easier as you understand more
tf_difficulty = [4, 3, 2, 1]     # Moderate learning curve
keras_difficulty = [1, 1, 2, 3]  # Easy start, harder to customize

ax1.plot(learning_stages, numpy_difficulty, 'b-o', label='NumPy', linewidth=2, markersize=8)
ax1.plot(learning_stages, tf_difficulty, 'r-s', label='TensorFlow', linewidth=2, markersize=8)
ax1.plot(learning_stages, keras_difficulty, 'g-^', label='Keras', linewidth=2, markersize=8)
ax1.set_ylabel('Difficulty Level')
ax1.set_title('Learning Difficulty by Experience Level')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 5)

# Development speed comparison
tasks = ['Simple\nNetwork', 'Complex\nArchitecture', 'Custom\nLayers', 'Research\nIdeas']
numpy_speed = [2, 1, 3, 4]    # Slow for simple, good for custom
tf_speed = [3, 4, 4, 3]       # Balanced
keras_speed = [5, 4, 2, 1]    # Fast for standard, slow for custom

x = np.arange(len(tasks))
width = 0.25

ax2.bar(x - width, numpy_speed, width, label='NumPy', color='blue', alpha=0.7)
ax2.bar(x, tf_speed, width, label='TensorFlow', color='red', alpha=0.7)
ax2.bar(x + width, keras_speed, width, label='Keras', color='green', alpha=0.7)
ax2.set_ylabel('Development Speed')
ax2.set_title('Development Speed by Task Type')
ax2.set_xticks(x)
ax2.set_xticklabels(tasks)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Performance comparison
dataset_sizes = ['Small\n(<1K)', 'Medium\n(1K-100K)', 'Large\n(100K+)', 'Huge\n(1M+)']
numpy_perf = [4, 3, 2, 1]     # Good for small, poor for large
tf_perf = [3, 4, 5, 5]        # Scales well
keras_perf = [3, 4, 5, 5]     # Same as TF (built on TF)

ax3.plot(dataset_sizes, numpy_perf, 'b-o', label='NumPy', linewidth=2, markersize=8)
ax3.plot(dataset_sizes, tf_perf, 'r-s', label='TensorFlow', linewidth=2, markersize=8)
ax3.plot(dataset_sizes, keras_perf, 'g-^', label='Keras', linewidth=2, markersize=8)
ax3.set_ylabel('Performance Level')
ax3.set_title('Performance by Dataset Size')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 6)

# Use case recommendations
use_cases = ['Learning\nML', 'Prototyping', 'Production', 'Research']
numpy_fit = [5, 2, 1, 4]      # Excellent for learning
tf_fit = [3, 4, 5, 4]         # Balanced
keras_fit = [2, 5, 4, 2]      # Great for prototyping

ax4.bar(x - width, numpy_fit, width, label='NumPy', color='blue', alpha=0.7)
ax4.bar(x, tf_fit, width, label='TensorFlow', color='red', alpha=0.7)
ax4.bar(x + width, keras_fit, width, label='Keras', color='green', alpha=0.7)
ax4.set_ylabel('Suitability Level')
ax4.set_title('Best Use Cases')
ax4.set_xticks(x)
ax4.set_xticklabels(use_cases)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 DECISION GUIDE")
print("=" * 40)
print("Choose NumPy when:")
print("  • Learning ML fundamentals")
print("  • Understanding algorithms deeply")
print("  • Implementing custom research ideas")
print("  • Small datasets and simple models")

print("\nChoose TensorFlow Low-Level when:")
print("  • Need control + automatic gradients")
print("  • Building custom training loops")
print("  • Complex model architectures")
print("  • Performance is critical")

print("\nChoose Keras when:")
print("  • Rapid prototyping")
print("  • Standard architectures")
print("  • Production deployment")
print("  • Team collaboration")

## 🎓 Summary and Recommendations

### 📊 What We Learned:

1. **All three approaches solve the same problem** but with different trade-offs
2. **NumPy gives you complete understanding** of what's happening
3. **TensorFlow provides automation** while maintaining control
4. **Keras maximizes productivity** for standard use cases

### 🏆 Performance Results:
- **Speed**: Keras ≈ TensorFlow > NumPy
- **Learning**: NumPy > TensorFlow > Keras
- **Productivity**: Keras > TensorFlow > NumPy
- **Flexibility**: NumPy ≈ TensorFlow > Keras

### 🚀 Your Learning Path Forward:

**Recommended Progression**:
1. **Master NumPy first** (you're doing this! ✅)
2. **Learn TensorFlow concepts** (automatic differentiation, variables)
3. **Use Keras for real projects** (rapid development)
4. **Return to low-level TensorFlow** when you need custom solutions

### 💡 Key Insights:

- **NumPy teaches you the "why"** - essential for deep understanding
- **TensorFlow teaches you the "how"** - industry-standard tools
- **Keras teaches you the "what"** - practical application

**Your NumPy foundation is invaluable!** It makes you a better ML practitioner because you understand what's happening under the hood. Keep building on it while exploring TensorFlow for practical applications.

### 🎯 Next Steps:
1. Continue with your NumPy implementations for learning
2. Try the next notebook: `03_Keras_Sequential_Models.ipynb`
3. Build more complex architectures with both approaches
4. Explore real-world datasets with TensorFlow/Keras

**Remember**: The best ML engineers understand both the theory (NumPy) and the tools (TensorFlow/Keras)! 🚀